In [1]:
'''
1. 将拼多多收入与订单合并
2. 对收入分别按订单汇总、按存货编码汇总
202405-202409
保留中间过程
'''

import easygui
import os
import pandas as pd
import re
import numpy as np



In [2]:
regax = r'（订单(?P<dingdanhao>\d{6}-\d{15})）'
pattern = re.compile(regax)
# mat = pattern.search(string)
def getDingdanhao(s):
    mat = pattern.search(s)
    if mat :
        s = mat.group('dingdanhao')
    else :
        s = ''
    return s

#所有收入
def chuliFnameShouru(fname):
    # fname_shouru = r"F:\a00nutstore\008\zw08\电商\拼多多\pdd-mall-bill-detail(100306881)_2024-09-30-07-09-49_3.csv"
    df_shouru0 = pd.read_csv(fname,header = 4,encoding = 'gb18030')
    df_shouru1 = df_shouru0[['商户订单号','发生时间','收入金额（+元）','支出金额（-元）','账务类型','备注','业务描述']]
    df_shouru1 = df_shouru1.dropna(thresh = 2)   #至少有三个缺失值 
    df_shouru1['商户订单号']= df_shouru1['商户订单号'].mask(df_shouru1['商户订单号'].isnull(),df_shouru1['备注'].map(getDingdanhao))
    df_shouru1['商户订单号'] = df_shouru1['商户订单号'].replace('','888888-888888888888888')
    df_shouru1.columns = ['dingdanhao', 'riqi', 'ru', 'chu', 'leixing', 'beizhu', 'miaoshu']
    df_shouru2 = df_shouru1.loc[df_shouru1.leixing != '提现']
    df_shouru2 = df_shouru2.assign(shouru = df_shouru2['ru'] + df_shouru2['chu'])
    return df_shouru2

dic = {'五': 5,
       '六': 6,
       '七': 7,
       '八': 8,
       '九': 9,
       '十': 10,
       '四': 4,
       '三': 3,
       '二': 2,
       '一': 1}


def addBen(string):
    # string = r'(81本)'
    pattern1 = r'(?P<ben>\d+)本'
    pattern2 = r'(?P<ben>[十九八七六五四三二一])本'
    regexp1 = re.compile(pattern1)
    mat1 = regexp1.search(string)
    regexp2 = re.compile(pattern2)
    mat2 = regexp2.search(string)
    if mat1:
        ben = int(mat1.group('ben'))
    else:
        if mat2:
            ben = dic.get(mat2.group('ben'))
        else:
            ben = 1
    return ben



In [3]:
#所有订单
path_dingdan = r'F:\a00nutstore\008\zw08\电商\拼多多\拼多多订单创业至20240930'
data_dingdan = []
for i in os.listdir(path_dingdan):
    j = os.path.join(path_dingdan,i)
    if  os.path.isfile(j) :
        df = pd.read_csv(j)
        data_dingdan.append(df)
    else :
        continue




def readDingdan(fname) :
    df_dingdan = pd.read_csv(fname)
    return df_dingdan

def chuliDingdan(df_dingdan0):
    df_dingdan1 = df_dingdan0[['订单号','发货时间','商家编码-规格维度','商家实收金额(元)','商品数量(件)','商品规格']]
    df_dingdan1.columns = ['dingdanhao','riqi','code','jiner','bao','guige']
    df_dingdan1[df_dingdan1['dingdanhao'].isnull()]
    df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)
    df_dingdan1 = df_dingdan1.assign(ben=df_dingdan1['bao'] * df_dingdan1['hanliang'])
    print(df_dingdan1.columns.to_list())
    df_dingdan2 = df_dingdan1[['dingdanhao','riqi','code','jiner','bao','guige','ben']]
    df_dingdan2 = df_dingdan2.set_index('dingdanhao')
    return df_dingdan2


df_dingdan0 = pd.concat(data_dingdan)
df_dingdan2 = chuliDingdan(df_dingdan0)
        
df_dingdan2.head()


['dingdanhao', 'riqi', 'code', 'jiner', 'bao', 'guige', 'hanliang', 'ben']


C:\Users\redda\AppData\Local\Temp\ipykernel_22316\637956375.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)


,riqi,code,jiner,bao,guige,ben
dingdanhao,,,,,,
240929-658935703962749,2024-09-30 11:14:43,EJDWB550-英高,7.84,1,"高级版,5本",5
240929-379762829722749,2024-09-30 11:14:43,EJDWB550-英初,7.83,1,"初级版,5本",5
240929-211738969641353,2024-09-30 11:14:43,N0581,7.85,1,B5五本（每本40张）,5
240929-611277893212149,2024-09-30 11:14:43,N0483,9.57,1,A5五本（每本80张）,5
240929-080425849263824,\t,N0481,4.27,1,A5三本（每本40张）,3


In [4]:
#拼多多收入
path = r"F:\a00nutstore\008\zw08\电商\拼多多\拼多多202410收入"
# os.chdir(path)
#莱新销售订单超5000条，不能一次导出，分三次导出，并分别存于同一文件下，先将它们合并
data = []
for i in os.listdir(path):
    j = os.path.join(path,i)
    if  os.path.isfile(j):
        df = chuliFnameShouru(j)
        data.append(df)
    else :
        continue


df_xiaoshou0 = pd.concat(data)
df_xiaoshou0.head()

,dingdanhao,riqi,ru,chu,leixing,beizhu,miaoshu,shouru
0,241025-546738092463927,2024-10-31 23:50:08,0.00,-0.02,技术服务费,-,0030002|技术服务费-基础技术服务费,-0.02
1,241025-546738092463927,2024-10-31 23:50:08,2.50,0.00,交易收入,-,0010002|交易收入-订单收入,2.50
2,241026-374761080362086,2024-10-31 23:45:09,4.22,0.00,交易收入,-,0010002|交易收入-订单收入,4.22
3,241026-374761080362086,2024-10-31 23:45:09,0.00,-0.03,技术服务费,-,0030002|技术服务费-基础技术服务费,-0.03
4,241028-499856259423117,2024-10-31 23:40:10,0.00,-0.02,技术服务费,-,0030002|技术服务费-基础技术服务费,-0.02


In [5]:
shouru_pivot =df_xiaoshou0.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)
result0 =pd.merge(shouru_pivot,df_dingdan2,left_index = True,right_index = True,how = 'left')
#空值填充为0
for i in ['chu','ru','shouru','jiner','bao','ben']:
    result0[i] = result0[i].fillna(0)
#code填充
result0['code'] = result0['code'].replace('\t',None)
result0['guige'] = result0['guige'].replace('\t',None)
result0['code'] = result0['code'].fillna('A8888')
result0['guige'] = result0['guige'].fillna('A8888')

#日期填充
result0['riqi'] = result0['riqi'].replace('\t',None)
result0['riqi'] = result0['riqi'].ffill()
result0['riqi'] = result0['riqi'].bfill()
result0.head()

C:\Users\redda\AppData\Local\Temp\ipykernel_22316\3571662054.py:1: FutureWarning: The provided callable <function sum at 0x00000224BCDCAD40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  shouru_pivot =df_xiaoshou0.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)


,chu,ru,shouru,riqi,code,jiner,bao,guige,ben
dingdanhao,,,,,,,,,
240816-369738401350356,-5.00,5.00,0.00,2024-08-17 12:17:36,N0784(300格),10.50,1.0,16K作文本(10本),10.0
240820-318221914410933,-2.97,0.02,-2.95,2024-08-21 09:25:29,N0782(双竖线),2.97,1.0,16K数学双线本(5本),5.0
240822-069247966781167,-1.00,0.00,-1.00,2024-08-22 11:28:57,N0784(300格),2.97,1.0,16K作文本(5本),5.0
240828-087503747422591,-20.02,0.00,-20.02,2024-08-28 14:18:56,N0785,20.02,1.0,"20本加厚大号,练习本",20.0
240831-486497328703337,-3.00,0.00,-3.00,2024-09-01 11:39:47,781（方格）+782（双竖线）+785,15.30,1.0,15本（语文+数学+练习各5本）,15.0


In [8]:
gp = result0.groupby('dingdanhao')
#将订单总金额按各产品的明细销售额进行分配
data = []
for k,v in gp:
    # print(k,v)
    mean = v.shouru.mean()
    total = v['jiner'].sum()
    # print('mean',mean)
    # print('total',total)
    v['result_jiner'] = v.shouru*(v.jiner)/total
    data.append(v)
result1 = pd.concat(data)
result1 = result1.reset_index()


In [9]:
result1 = result1.rename({'riqi': '日期',
 'dingdanhao': '订单号',
 'ru': '收入',
 'chu': '支出',
 'shouru': '净收入',
 'code': '存货编码',
 'jiner': '订单金额',
 'bao': '包',
   'ben': '本',
   'guige': '商品规格',
                          
 'result_jiner': '最终金额'},axis = 1)
result1  = result1[['日期', '订单号', '收入', '支出', '净收入', '存货编码', '订单金额', '包','本', '最终金额','商品规格']]
result1 ['最终金额']= np.where(result1 ['最终金额'].isnull(),result1 ['净收入'],result1['最终金额'])
# result1.to_excel('分配后15.xlsx',index=False)

In [10]:
total = result1.sum().T.to_frame().T
total.日期  = '合计'
total.订单号  = ''
total.存货编码  = ''
total.商品规格  = ''
result2 = pd.concat([result1,total])
result2.to_excel('拼多多202410净收入明细.xlsx',index = False)

In [ ]:
# result1 = result1.rename({'':'888888-888888888888888'})
# result0.to_excel('分配前outer-2-8.xlsx')
# df_dingdan2.to_excel('订单号-8.xlsx')


In [ ]:
df00 = pd.read_excel(r"F:\repos\fisha\拼多多202409净收入明细.xlsx")
df00.columns.to_list()

In [ ]:
result1.columns.to_list()

In [ ]:
dic = dict(zip(['riqi','dingdanhao','ru','chu',  'shouru',  'code', 'jiner', 'shuliang', 'result_jiner'],
['日期', '订单号', '收入', '支出', '净收入', '存货编码', '订单金额', '数量', '最终金额']))
dic

In [ ]:
df11 = pd.read_clipboard()
df11

In [ ]:
import numpy as np
df11['最终金额']= np.where(any(df11['最终金额'].isnull()*df11['最终金额'] == '' == 0),df11['收入'],df11['最终金额'])
df11['最终金额']

In [ ]:
df11

In [ ]:
df11

In [ ]:
df22 = df11.copy()
df22

In [ ]:
df22['最终金额']= np.where(df22['最终金额'].isnull(),df22['净收入'],df22['最终金额'])
df22

In [ ]:
df22['最终金额'].isnull()

In [ ]:
df23 = df22.sum().T.to_frame().T
df23.日期  = '合计'
df23.订单号  = ''
df23.存货编码  = ''
df23

In [ ]:
df24 = df22.sum().str.replace('日期','')

In [ ]:
type(df22.sum())

In [ ]:
df24
